In [0]:
# ARGUMENTOS DESTINADOS A CONEXÃO COM O DATABASE
PARAMS = {
    "DRIVER" : "org.postgresql.Driver",
    "DB" : "ecom1692155331663giqokzaqmuqlogbu",
    "USER" : "eolowynayhvayxbhluzaqxfp@psql-mock-database-cloud",
    "PWD" : "hdzvzutlssuozdonhflhwyjm",
    "HOST" : "psql-mock-database-cloud.postgres.database.azure.com",
    "PORT" : 5432,
}
URL = f"jdbc:postgresql://{PARAMS['HOST']}:{PARAMS['PORT']}/{PARAMS['DB']}"

LIST_TABLES = ['customers', 
               'employees', 
               'offices', 
               'orderdetails', 
               'orders', 
               'payments', 
               'product_lines', 
               'products']

In [0]:
# CONEXÃO E EXTRAÇÃO DO DATABASE
def extract_data(driver: str, url: str, user: str, pwd: str, table_name: str):
    try:
        table = (spark.read.format("jdbc").option("driver", driver).option("url", url).option("dbtable", table_name).option("user", user).option("password", pwd).load())
        return table
    except Exception as e:
        print(e)

In [0]:
# CONSTRUÇÃO DO ARQUIVO PARQUET
def create_file(driver: str, url: str, user: str, pwd: str, list_tables: list[str]):
    try:
        for table_name in list_tables:
            table = extract_data(driver, url, user, pwd, table_name)
            table.write.mode("overwrite").format("parquet").save(f"/FileStore/tables/{table_name}")
        print("DADOS EXTRAÍDOS!")
        print("ARQUIVOS PARQUET'S GERADOS")
    except Exception as e:
        print(e)

In [0]:
# LEITURA DO ARQUIVO PARQUET
def read_parquet(table_name: str):
    try:
        files_to_tables = spark.read.parquet(f"/FileStore/tables/{table_name}/*.parquet")
        return files_to_tables
    except Exception as e:
        print(e)
    

In [0]:
# CONSTRUÇÃO DA TABELA NO DATABRICKS
def create_table(list_tables: list[str]):
    try:
        for table_name in list_tables:
            data = read_parquet(table_name)
            data.createOrReplaceTempView(f"tmp_{table_name}")
        print("ARQUIVOS PARQUET LIDOS")
    except Exception as e:
        print(e)


In [0]:
# QUERY DESTINADA A ATUALIZAÇÃO DE ALGUMA TABELA
def update_table():
    try:
        spark.sql("--UPDATE <table_name> SET <coluna_que_sera_alterada> = <new_value> WHERE <reference_colunm> = <value>")
        print("UPDATE REALIZADO COM SUCESSO!")
    except Exception as e:
        print(e)

In [0]:
# QUERY DESTINADA A DELEÇÃO DE ALGUMA TABELA
def delete_table():
    try:
        spark.sql("--DELETE FROM <table_name> WHERE <reference_colunm> = <value>")
        print("DELETE REALIZADO COM SUCESSO!")
    except Exception as e:
        print(e)

In [0]:
# QUERY DESTINADA A PROJEÇÃO DOS VENDEDORES DO JAPÃO
def vendedores():
    try:
        vendedoresJapan = spark.sql("SELECT first_name as Nome, \
                            last_name as Sobrenome, \
                            regexp_extract(email, '^.*@(.*)$', 1) as Email \
        FROM tmp_employees \
        INNER JOIN tmp_offices ON tmp_employees.office_code = tmp_offices.office_code \
        WHERE tmp_employees.job_Title = 'Sales Rep' AND tmp_offices.country = 'Japan'")
        vendedoresJapan.write.format("delta").mode("overwrite").save("/tmp/delta/vendedores")
        print('IDENTIFICADO VENDEDORES DO JAPÃO')
    except Exception as e:
        print(e)


In [0]:
# QUERY DESTINADA AO LEVANTAMENTO DO MAIOR FATURAMENTO DE 2005
def faturamento():
    try:
        maiorFaturamento = spark.sql("select sum(tmp_payments.amount) as Faturamento, tmp_products.product_line as Linha_de_Produtos \
        from tmp_orders \
        INNER JOIN tmp_payments \
        ON tmp_orders.customer_number = tmp_payments.customer_number \
        INNER JOIN tmp_orderdetails \
        ON tmp_orders.order_number = tmp_orderdetails.order_number \
        INNER JOIN tmp_products \
        ON tmp_orderdetails.product_code = tmp_products.product_code \
        WHERE tmp_orders.status = 'Shipped' and year(tmp_orders.order_date) = 2005 \
        group by 2 order by Faturamento desc limit 1")
        maiorFaturamento.write.format("delta").mode("overwrite").save("/tmp/delta/faturamento")
        print('LEVANTADO O MAIOR FATURAMENTO')
    except Exception as e:
        print(e)

In [0]:
# QUERY DESTINADA A IDENTIFICAÇÃO DO PAÍS QUE TEVE A MAIOR QUANTIDADE DE ITENS CANCELADOS 
def itens_cancelados():
    try:
        itensCancelados = spark.sql("select tmp_offices.country as Pais, SUM(tmp_orderdetails.quantity_ordered)as Itens_Cancelados from tmp_orders \
        INNER JOIN tmp_orderdetails \
        ON tmp_orders.order_number = tmp_orderdetails.order_number \
        inner join tmp_customers \
        ON tmp_orders.customer_number = tmp_customers.customer_number \
        INNER JOIN tmp_employees \
        ON tmp_customers.sales_rep_employee_number = tmp_employees.employee_number \
        INNER JOIN tmp_offices \
        ON tmp_employees.office_code = tmp_offices.office_code \
        where tmp_orders.status = 'Cancelled' \
        GROUP BY 1 ORDER BY 2 DESC LIMIT 1")
        itensCancelados.write.format("delta").mode("overwrite").save("/tmp/delta/itensCancelados")
        
        print('IDENTIFICADO O PAÍS COM A MAIOR QUANTIDADE DE ITENS CANCELADOS')
    except Exception as e:
        print(e)

In [0]:
# MÉTODO MAIN ONDE STARTAR A EXECUÇÃO DAS FUNÇÕES PRINCIPAIS
def main(driver: str, url: str, user: str, pwd: str, list_tables: list[str]):
    create_file(driver, url, user, pwd, list_tables)
    create_table(list_tables)
    print("TABELAS CONSTRUÍDAS")
    #update_table()
    #delete_table()
    vendedores()
    faturamento()
    itens_cancelados()
    print("MÉTODO MAIN EXECUTADO")

In [0]:
# DISPARO DA FUNÇÃO MAIN, PASSANDO OS VALORES
if __name__ == "__main__":
    main(PARAMS['DRIVER'], 
         URL, 
         PARAMS['USER'], 
         PARAMS['PWD'], 
         LIST_TABLES)

DADOS EXTRAÍDOS!
ARQUIVOS PARQUET'S GERADOS
ARQUIVOS PARQUET LIDOS
TABELAS CONSTRUÍDAS
IDENTIFICADO VENDEDORES DO JAPÃO
LEVANTADO O MAIOR FATURAMENTO
IDENTIFICADO O PAÍS COM A MAIOR QUANTIDADE DE ITENS CANCELADOS
MÉTODO MAIN EXECUTADO
